### SCRAPING

In [ ]:
# Reddit mining using PRAW
import praw

reddit = praw.Reddit(
  client_id='6NE27-qV7tGX07FuBfik3w',
  client_secret='fauqCWKne1OXmmtzPIABkNW57DFyZA',
  user_agent='Comment scraper'
)

In [ ]:
import json
import re
import html
from langdetect import detect, LangDetectException

subreddit_names = ['Anxiety', 'mentalhealth']
posts_data = []
total_comments = 0
target_comments = 5000
mod_keywords = ['announcement', 'rules', 'mod', 'moderator', 'admin', 'clarification', 'faq', 'meta', 'update', 'policy']

def is_mod_post(post):
  if post.distinguished in ['moderator', 'admin']:
    return True
  text = (post.title + " " + post.selftext).lower()
  return any(keyword in text for keyword in mod_keywords)

def is_mod_comment(comment):
  if comment.distinguished in ['moderator', 'admin']:
    return True
  text = comment.body.lower()
  return any(keyword in text for keyword in mod_keywords)

def clean_text(text):
  if not text:
    return ""
  text = html.unescape(text)                                                  # Decode HTML entities
  text = re.sub(r'\[deleted\]|\[removed\]', '', text, flags = re.IGNORECASE)  # remove [deleted]/[removed]
  text = re.sub(r'http\S+|www\S+', '', text)                                  # Remove URLs
  text = re.sub(r'&\w+;', '', text)                                           # Remove encoded HTML symbols
  text = re.sub(r'u/\w+|r/\w+', '', text)                                     # Remove mentions
  text = re.sub(r'>.*\n?', '', text)                                          # Remove blockquotes
  text = re.sub(r'\*+', '', text)                                             # Remove markdown asterisks
  text = re.sub(r'[\r\n\t]', ' ', text)                                       # Normalize whitespace
  text = re.sub(r'[^a-zA-Z\s\.,!?\'":;()\[\]{}\-]', '', text)                 # Remove numbers, unicode, & symbols
  text = re.sub(r'[^\x00-\x7F]+', '', text)                                   # Remove non-ASCII characters
  text = re.sub(r'\s+([.,!?;:])', r'\1', text)                                # Remove spaces before punctuation
  text = re.sub(r'([!?])\1{2,}', r'\1\1\1', text)                             # Limit repeated punctuation to max 3
  text = re.sub(r'(\.\s*){2,}', '...', text)                                  # Normalize ellipses
  text = re.sub(r'\.{4,}', '...', text)                                       # Normalize ellipses
  text = re.sub(r'\s{2,}', ' ', text)                                         # Collapse multiple spaces
  return text.strip().lower()

def is_english(text):
  try:
    if len(text) < 20:
      return False
    return detect(text) == 'en'
  except LangDetectException:
    return False

for sub_name in subreddit_names:
  sub_comments = 0
  
  for submission in reddit.subreddit(sub_name).new(limit = None):
    if sub_comments >= target_comments / len(subreddit_names):
      break

    if is_mod_post(submission):
      continue

    submission.comments.replace_more(limit = 0)
    comments = []

    for comment in submission.comments.list():
      if is_mod_comment(comment):
        continue

      clean_body = clean_text(comment.body)

      if not is_english(clean_body):
        continue

      comments.append({
        'comment_id': comment.id,
        'body': clean_body,
        'author': comment.author.name if comment.author else "[deleted]",
        'author_role': comment.distinguished,
        'score': comment.score,
        'created_utc': comment.created_utc,
        'is_submitter': comment.is_submitter,
        'parent_id': comment.parent_id,
        'permalink': comment.permalink
      })

    if comments:
      sub_comments += len(comments)
      total_comments += len(comments)

      post = {
        'post_id': submission.id,
        'title': clean_text(submission.title),
        'selftext': clean_text(submission.selftext),
        'author': submission.author.name if submission.author else "[deleted]",
        'author_role': submission.distinguished,
        'score': submission.score,
        'upvote_ratio': submission.upvote_ratio,
        # 'url': submission.url,
        'created_utc': submission.created_utc,
        'num_comments': len(comments),
        'subreddit': submission.subreddit.display_name,
        'flair': submission.link_flair_text,
        'is_self': submission.is_self,
        'nsfw': submission.over_18,
        'permalink': submission.permalink,
        'comments': comments
      }
      posts_data.append(post)

  print(f"{sub_comments} comments collected from r/{sub_name}")

with open('raw_posts.json', 'w', encoding='utf-8') as f:
  json.dump(posts_data, f, indent = 2, ensure_ascii=False)

print(f"\nScraped {len(posts_data)} posts with {total_comments} comments.")

### DATA CLEANING

In [ ]:
import json

with open('raw_posts.json', 'r', encoding='utf-8') as f:
  raw_data = json.load(f)

In [ ]:
from datetime import datetime, timezone

cleaned_data = []
total_cleaned_posts = 0
total_cleaned_comments = 0
mod_keywords = ['announcement', 'rules', 'mod', 'moderator', 'admin', 'clarification', 'faq', 'meta', 'update', 'policy']

def format_datetime(utc_timestamp):
  return datetime.fromtimestamp(utc_timestamp, tz=timezone.utc).isoformat()

def is_meaningful(text):
  if len(text) < 20:                      # Remove short comments
    return False
  if not any(c.isalpha() for c in text):  # Must contain alphabetic characters
    return False
  if len(text) > 2000:                    # remove very long comments 
    return False
  return True

for post in raw_data:
  cleaned_post = {
    'post_id': post['post_id'],
    'title': post['title'],
    'selftext': post['selftext'],
    'author': post['author'],
    'author_role': post['author_role'],
    'score': post['score'],
    'upvote_ratio': post['upvote_ratio'],
    'created_utc': format_datetime(post['created_utc']),
    'num_comments': 0,
    'subreddit': post['subreddit'],
    'flair': post['flair'],
    'is_self': post['is_self'],
    'nsfw': post['nsfw'],
    'permalink': post['permalink'],
    'comments': []
  }

  for comment in post['comments']:
    if comment['body'] == "":
      continue
    if comment['author'] == "[deleted]":
      continue
    if not is_meaningful(comment['body']):
      continue

    cleaned_comment = {
      'comment_id': comment['comment_id'],
      'body': comment['body'],
      'author': comment['author'],
      'author_role': comment['author_role'],
      'score': comment['score'],
      'created_utc': format_datetime(comment['created_utc']),
      'is_submitter': comment['is_submitter'],
      'parent_id': comment['parent_id'],
      'permalink': comment['permalink']
    }
    cleaned_post['comments'].append(cleaned_comment)

  if cleaned_post['comments']:
    cleaned_post['num_comments'] = len(cleaned_post['comments'])
    total_cleaned_posts += 1
    total_cleaned_comments += len(cleaned_post['comments'])
    cleaned_data.append(cleaned_post)

with open('cleaned_posts.json', 'w', encoding='utf-8') as f:
  json.dump(cleaned_data, f, indent = 2, ensure_ascii = False)

print(f"Data cleaned. {total_cleaned_posts} posts remaining with {total_cleaned_comments} comments.")

### SQL LOADING

In [ ]:
import psycopg2

conn = psycopg2.connect(
  host = "127.0.0.1",
  user = "postgres",
  password = "SqlPassword",
  dbname = "reddit_data",
  port = "5432"
)
cur = conn.cursor()

In [ ]:
import json

with open('cleaned_posts.json', 'r', encoding = 'utf-8') as f:
  data = json.load(f)

In [ ]:
from datetime import datetime, timezone

def unique_topic(flair):
  cur.execute("""
    INSERT INTO dim_topic (flair)
    VALUES (%s)
    ON CONFLICT (flair) DO NOTHING
    RETURNING topic_id;
  """, (flair,))
  result = cur.fetchone()
  if not result:
    cur.execute("SELECT topic_id FROM dim_topic WHERE flair = %s;", (flair,))
    result = cur.fetchone()
  return result[0]

def unique_date_id(dt):
  cur.execute("""
    INSERT INTO dim_date (full_date, year, quarter, month, day, weekday)
    VALUES (%s, %s, %s, %s, %s, %s)
    ON CONFLICT (full_date) DO NOTHING
    RETURNING date_id;
  """, (dt.date(), dt.year, ((dt.month - 1) // 3 + 1), dt.month, dt.day, dt.strftime('%A')))
  result = cur.fetchone()
  if not result:
    cur.execute("SELECT date_id FROM dim_date WHERE full_date = %s;", (dt.date(),))
    result = cur.fetchone()
  return result[0]

for post in data:
  cur.execute("""
    INSERT INTO dim_posts (post_id, title, selftext, author, author_role, score, upvote_ratio, created_utc, num_comments, subreddit, is_self, nsfw, permalink)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, (post['post_id'], post['title'], post['selftext'], post['author'], post['author_role'], post['score'], post['upvote_ratio'], post['created_utc'], post['num_comments'], post['subreddit'], post['is_self'], post['nsfw'], post['permalink'])
  )

  topic_id = unique_topic(post['flair'])

  for comment in post['comments']:
    cur.execute("""
      INSERT INTO dim_comment (comment_id, author, author_role, body, score, created_utc, is_submitter, parent_id, permalink)
      VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
      """, (comment['comment_id'], comment['author'], comment['author_role'], comment['body'], comment['score'], comment['created_utc'], comment['is_submitter'], comment['parent_id'], comment['permalink'])
    )

    dt = datetime.fromisoformat(comment['created_utc'])
    date_id = unique_date_id(dt)

    cur.execute("""
      INSERT INTO fact_table (comment_id, post_id, topic_id, date_id, is_submitter, comment_score, comment_length)
      VALUES (%s, %s, %s, %s, %s, %s, %s);
      """, (comment['comment_id'], post['post_id'], topic_id, date_id, comment['is_submitter'], comment['score'], len(comment['body']))
    )

conn.commit()
cur.close()
conn.close()

### TOPIC MODELING

In [ ]:
import psycopg2
import pandas as pd
import spacy

# Load spaCy English model
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

conn = psycopg2.connect(
  host = "127.0.0.1",
  user = "postgres",
  password = "SqlPassword",
  dbname = "reddit_data",
  port = "5432"
)
cur = conn.cursor()

# Sample: Load comments
df = pd.read_sql_query("SELECT comment_id, body FROM dim_comment;", conn)
df.dropna(subset=["body"], inplace=True)

# Lemmatize and clean
def preprocess(text):
  doc = nlp(text)
  return " ".join([
    token.lemma_ for token in doc 
    if not token.is_stop and token.is_alpha and token.pos_ != "PRON"
  ])

# Apply preprocessing
df["cleaned_body"] = df["body"].apply(preprocess)
documents = df["cleaned_body"].tolist()

In [23]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", nr_topics=3)  # Reduce to 3 topics
topics, probs = topic_model.fit_transform(documents)

In [25]:
# Inspect topic keywords
topic_info = topic_model.get_topic_info()
print(topic_info)

# Example: get keywords for each topic
for i in range(3):
  print(f"Topic {i} keywords:", topic_model.get_topic(i - 1))


   Topic  Count                         Name  \
0     -1   3800    -1_feel_like_anxiety_help   
1      0   1080     0_like_people_know_think   
2      1    110  1_anxiety_attack_panic_feel   

                                      Representation  \
0  [feel, like, anxiety, help, thing, time, know,...   
1  [like, people, know, think, good, thank, life,...   
2  [anxiety, attack, panic, feel, like, help, tim...   

                                 Representative_Docs  
0  [ill honest experience anxiety shock deeply bo...  
1  [hey happen time feel lose disconnected time g...  
2  [bad thing anxiety rid severe case chronic anx...  
Topic 0 keywords: [('feel', np.float64(0.05193557742878411)), ('like', np.float64(0.049319934840007115)), ('anxiety', np.float64(0.042647111034908626)), ('help', np.float64(0.04087425041857313)), ('thing', np.float64(0.035242321498194645)), ('time', np.float64(0.035112777569198075)), ('know', np.float64(0.03344455598681058)), ('people', np.float64(0.0329696037